In [19]:
#@title 🚗 Dashboard Elegante de Costos de Movilidad v13.0
#@markdown Mueva los deslizadores y seleccione una base de comparación para analizar los indicadores clave (KPIs).

import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as patches

# --- Título Principal ---
display(HTML("<h1 style='text-align:center; font-family:sans-serif; color:#2c3e50;'>Dashboard de Eficiencia Vehicular</h1>"))

# Estilo profesional para las gráficas
plt.style.use('seaborn-v0_8-whitegrid')

# --- Controles Interactivos ---
style = {'description_width': 'initial'}
# Precios
tarifa_electrica = widgets.FloatSlider(value=0.709, min=0.5, max=1.5, step=0.001, readout_format='.3f', description='Tarifa Eléctrica (S/kWh)', style=style)
precio_gnv = widgets.FloatSlider(value=1.680, min=1.0, max=3.0, step=0.05, description='Precio GNV (S/m³)', style=style)
precio_glp = widgets.FloatSlider(value=7.50, min=5.0, max=12.0, step=0.10, description='Precio GLP (S/galón)', style=style)
precio_gasolina = widgets.FloatSlider(value=17.00, min=12.0, max=25.0, step=0.10, description='Precio Gasolina (S/galón)', style=style)

# Rendimientos
rendimiento_bev = widgets.FloatSlider(value=16.0, min=10, max=25, step=0.5, description='Rend. BEV (kWh/100km)', style=style)
rendimiento_gnv = widgets.FloatSlider(value=12.5, min=8, max=25, step=0.5, description='Rend. GNV (km/m³)', style=style)
rendimiento_glp = widgets.FloatSlider(value=35.0, min=20, max=50, step=1, description='Rend. GLP (km/galón)', style=style)
rendimiento_gasol = widgets.FloatSlider(value=40.0, min=25, max=60, step=1, description='Rend. Gasolina (km/galón)', style=style)

# Distancia
km_recorridos = widgets.IntSlider(value=350, min=50, max=1000, step=10, description='Distancia del Viaje (km)', style=style)

# Menú de selección para la comparación
base_comparacion = widgets.Dropdown(
    options=['Gasolina', 'GLP', 'GNV', 'BEV'],
    value='Gasolina',
    description='Base de Comparación:',
    style=style
)

# --- Función principal que actualiza todo ---
def calcular_y_mostrar(tarifa, gnv, glp, gasolina, rend_bev, rend_gnv, rend_glp, rend_gasol, km, base_comp):
    # Cálculos de costos
    costos = {
        'BEV': km * (rend_bev / 100) * tarifa,
        'GNV': km * (1 / rend_gnv) * gnv,
        'GLP': km * (1 / rend_glp) * glp,
        'Gasolina': km * (1 / rend_gasol) * gasolina
    }

    # --- GRÁFICO PRINCIPAL: Costo Total del Viaje ---
    fig1, ax1 = plt.subplots(figsize=(10, 4))
    labels_main = [c[0] for c in sorted(costos.items(), key=lambda item: item[1])]
    costs_total_main = [c[1] for c in sorted(costos.items(), key=lambda item: item[1])]
    colors_pro = {'BEV':'#0077b6', 'GNV':'#00b4d8', 'GLP':'#fca311', 'Gasolina':'#e74c3c'}
    bar_colors = [colors_pro[label] for label in labels_main]

    bars1 = ax1.barh(labels_main, costs_total_main, color=bar_colors, alpha=0.9)
    ax1.set_title(f'Ranking de Costo Total para Recorrer {km} km', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Costo Total (Soles)', fontsize=12)
    ax1.spines['top'].set_visible(False); ax1.spines['right'].set_visible(False)

    for bar in bars1:
        xval = bar.get_width()
        ax1.text(xval + (max(costs_total_main)*0.01), bar.get_y() + bar.get_height()/2, f'S/ {xval:.2f}', va='center', fontsize=11, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # --- GRÁFICO PROFESIONAL: Panel de Control de Rendimiento ---
    fig2, axs = plt.subplots(1, 4, figsize=(12, 3.5))
    fig2.suptitle('Panel de Control de Rendimiento', fontsize=18, fontweight='bold')

    costo_base_ref = costos[base_comp]
    costo_100km_max = max( (100 * (1/rend_gasol) * gasolina), (100 * (1/rend_glp) * glp), (100 * (1/rend_gnv) * gnv), (100 * (rend_bev/100) * tarifa) )

    tecnologias = {'BEV': (rend_bev/100)*tarifa, 'GNV':(1/rend_gnv)*gnv, 'GLP':(1/rend_glp)*glp, 'Gasolina':(1/rend_gasol)*gasolina}

    for ax, (tech, costo_km) in zip(axs, tecnologias.items()):
        costo_100km = costo_km * 100
        ahorro_vs_base = (costo_base_ref - costos[tech]) / costo_base_ref if tech != base_comp else 0

        norm_cost = costo_100km / costo_100km_max
        color_indicador = plt.cm.RdYlGn_r(norm_cost)

        ax.set_xticks([]); ax.set_yticks([])
        ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False); ax.spines['bottom'].set_visible(False)

        # Se añade un cuadro con bordes redondeados y SIN borde
        ax.add_patch(patches.FancyBboxPatch(
            (0.0, 0.0), 1, 1,
            boxstyle="round,pad=0,rounding_size=0.1",
            facecolor='#f0f0f0',
            edgecolor='none',  # CAMBIO AQUÍ: Se quita el borde
            transform=ax.transAxes,
            zorder=-1)
        )

        # Jerarquía de fuentes ajustada
        ax.text(0.5, 0.88, tech, ha='center', va='center', fontsize=18, fontweight='bold', color=colors_pro[tech])
        ax.text(0.5, 0.65, f'S/ {costos[tech]:.2f}', ha='center', va='center', fontsize=20, fontweight='bold')
        ax.text(0.5, 0.55, f'Costo del Viaje ({km} km)', ha='center', va='center', fontsize=9, color='grey')

        ax.text(0.5, 0.38, f'S/ {costo_100km:.2f}', ha='center', va='center', fontsize=14, fontweight='bold')
        ax.text(0.5, 0.30, 'Costo / 100 km', ha='center', va='center', fontsize=9, color='grey')

        ahorro_color = '#2ecc71' if ahorro_vs_base > 0 else ('#e74c3c' if ahorro_vs_base < 0 else 'black')
        ax.text(0.5, 0.15, f'{ahorro_vs_base:.0%}', ha='center', va='center', fontsize=15, fontweight='bold', color=ahorro_color)
        ax.text(0.5, 0.08, f'Ahorro vs {base_comp}', ha='center', va='center', fontsize=9, color='grey')

        ax.add_patch(patches.Circle((0.15, 0.88), 0.07, facecolor=color_indicador, transform=ax.transAxes))

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.show()

    # --- TABLA DE AHORROS ---
    df_table = pd.DataFrame(index=costos.keys(), columns=costos.keys(), dtype=float)
    for row_tech in costos:
        for col_tech in costos:
            if row_tech != col_tech:
                ahorro = (costos[col_tech] - costos[row_tech]) / costos[col_tech]
                df_table.loc[row_tech, col_tech] = ahorro

    styled_df = df_table.style.format("{:.0%}", na_rep='-').set_caption("<b>Matriz de Ahorro: Fila vs. Columna (%)</b>").background_gradient(cmap='RdYlGn', axis=None, vmin=-1, vmax=1).set_properties(**{'text-align': 'center'})

    display(styled_df)

# --- Organización y Visualización de Controles ---
output = widgets.interactive_output(calcular_y_mostrar, {
    'tarifa': tarifa_electrica, 'gnv': precio_gnv, 'glp': precio_glp, 'gasolina': precio_gasolina,
    'rend_bev': rendimiento_bev, 'rend_gnv': rendimiento_gnv, 'rend_glp': rendimiento_glp, 'rend_gasol': rendimiento_gasol,
    'km': km_recorridos, 'base_comp': base_comparacion
})

# --- CAMBIO AQUÍ: Controles clave resaltados ---
# 1. Layout para resaltar
highlight_layout = widgets.Layout(
    background='#e7f3fe',
    border='2px solid #aed6f1',
    padding='10px',
    margin='10px 0px'
)

# 2. Cuadrícula para los parámetros estándar
grid = widgets.GridspecLayout(4, 2, width='100%')
grid[0, 0] = tarifa_electrica; grid[1, 0] = precio_gnv; grid[2, 0] = precio_glp; grid[3, 0] = precio_gasolina
grid[0, 1] = rendimiento_bev; grid[1, 1] = rendimiento_gnv; grid[2, 1] = rendimiento_glp; grid[3, 1] = rendimiento_gasol

# 3. Cajas con estilo para los controles a destacar
distancia_box = widgets.Box([km_recorridos], layout=highlight_layout)
comparacion_box = widgets.Box([base_comparacion], layout=highlight_layout)

# 4. Agrupar todo en un contenedor vertical
controls_container = widgets.VBox([grid, distancia_box, comparacion_box])

# Mostrar todo en pantalla
display(controls_container, output)

Output()